# Udacity A/B Testing 

- **Goal: Increase free trial completion rate.**
- Experiment: Pop up that asks if student has >5 hours per week to commit.
    - If yes: continue with free trial (paid version for 14 days).
    - If no, page goes back to free version.
 
- Success Metrics:
    - Free trial Completion rate
    - Free trial Retention rate
    - Free trial Conversion rate (to paid)
    - Student satisfaction rates
    - Teacher satisfaction rates
- Guardrail Metrics:
    - Avg Daily/Weekly number of page views on course page
    - Avg Daily/Weekly clicks on free trial button

In [89]:
import math as mt
import numpy as np
import pandas as pd
from scipy.stats import norm
import os
from datetime import datetime
from scipy import stats

# Baseline Metrics

In [68]:
pd.set_option('display.width', None)

In [69]:
df_basevals = pd.read_csv(os.path.abspath("data/baseline_vals.csv"),
                          index_col=False,
                          header = None, 
                          names = ['metric','baseline_val'])
df_basevals.metric = df_basevals.metric.map(lambda x: x.lower())
display(df_basevals)

,metric,baseline_val
0,unique cookies to view page per day:,40000.000000
1,"unique cookies to click ""start free trial"" per...",3200.000000
2,enrollments per day:,660.000000
3,"click-through-probability on ""start free trial"":",0.080000
4,"probability of enrolling, given click:",0.206250
5,"probability of payment, given enroll:",0.530000
6,"probability of payment, given click",0.109313


In [70]:
# Defintions:
    # {"# Eyeballs: Unique Visitors per Day": 40000,
    # "# Clicks: Unique Clicks on the Free Trial Button per Day": 3200,
    # "CTR: # Clicks / # Eyeballs ":0.08,
    # "# Conversions: Number of Free Trial Enrollments per Day": 660,
    # "Free Trial Gross Conversion Rate: Total Free Trial Enrollments/Total Clicks": 0.20625,
    # "Paid User Net Conversion Rate from Free Trial Users": 0.53,
    # "Paid User Gross Conversion Rate: Total Paid Users/Total Clicks": 0.109313}

In [71]:
eyeballs = 40000
clicks = 3200
ctr = 0.08
n_conversions = 660
free_trial_conversion = 0.20625
paid_user_conversion_from_free_trial = 0.53
paid_user_conversion = 0.0075

# Define MDEs

In [72]:
mde_eyeballs = 40000*0.1
mde_clicks = 3200*0.1
mde_ctr = 0.01
mde_free_trial_conversion = 0.01
mde_paid_user_conversion_from_free_trial = 0.01
mde_paid_user_conversion = 0.0075

# Experiment

In [73]:
df_control = pd.read_csv(os.path.abspath("data/Final Project Results - Control.csv"))
df_experiment = pd.read_csv(os.path.abspath("data/Final Project Results - Experiment.csv"))
df_experiment

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0
5,"Thu, Oct 16",9500,788,129.0,61.0
6,"Fri, Oct 17",9088,780,127.0,44.0
7,"Sat, Oct 18",7664,652,94.0,62.0
8,"Sun, Oct 19",8434,697,120.0,77.0
9,"Mon, Oct 20",10496,860,153.0,98.0


# Cleaning

In [74]:
# Clean
dfs = [df_control, df_experiment]
for i in dfs:
    # Assuming the year is the current year
    current_year = datetime.now().year - 1
    i.columns = i.columns.str.lower()
    print(i['date'] + ', ' + str(current_year))
    i['date'] = pd.to_datetime(i['date'] + ', ' + str(current_year), format='%a, %b %d, %Y')

display(df_control)

0     Sat, Oct 11, 2023
1     Sun, Oct 12, 2023
2     Mon, Oct 13, 2023
3     Tue, Oct 14, 2023
4     Wed, Oct 15, 2023
5     Thu, Oct 16, 2023
6     Fri, Oct 17, 2023
7     Sat, Oct 18, 2023
8     Sun, Oct 19, 2023
9     Mon, Oct 20, 2023
10    Tue, Oct 21, 2023
11    Wed, Oct 22, 2023
12    Thu, Oct 23, 2023
13    Fri, Oct 24, 2023
14    Sat, Oct 25, 2023
15    Sun, Oct 26, 2023
16    Mon, Oct 27, 2023
17    Tue, Oct 28, 2023
18    Wed, Oct 29, 2023
19    Thu, Oct 30, 2023
20    Fri, Oct 31, 2023
21     Sat, Nov 1, 2023
22     Sun, Nov 2, 2023
23     Mon, Nov 3, 2023
24     Tue, Nov 4, 2023
25     Wed, Nov 5, 2023
26     Thu, Nov 6, 2023
27     Fri, Nov 7, 2023
28     Sat, Nov 8, 2023
29     Sun, Nov 9, 2023
30    Mon, Nov 10, 2023
31    Tue, Nov 11, 2023
32    Wed, Nov 12, 2023
33    Thu, Nov 13, 2023
34    Fri, Nov 14, 2023
35    Sat, Nov 15, 2023
36    Sun, Nov 16, 2023
Name: date, dtype: object
0     Sat, Oct 11, 2023
1     Sun, Oct 12, 2023
2     Mon, Oct 13, 2023
3     Tue, Oct

,date,pageviews,clicks,enrollments,payments
0,2023-10-11,7723,687,134.0,70.0
1,2023-10-12,9102,779,147.0,70.0
2,2023-10-13,10511,909,167.0,95.0
3,2023-10-14,9871,836,156.0,105.0
4,2023-10-15,10014,837,163.0,64.0
5,2023-10-16,9670,823,138.0,82.0
6,2023-10-17,9008,748,146.0,76.0
7,2023-10-18,7434,632,110.0,70.0
8,2023-10-19,8459,691,131.0,60.0
9,2023-10-20,10667,861,165.0,97.0


In [75]:
df_experiment

,date,pageviews,clicks,enrollments,payments
0,2023-10-11,7716,686,105.0,34.0
1,2023-10-12,9288,785,116.0,91.0
2,2023-10-13,10480,884,145.0,79.0
3,2023-10-14,9867,827,138.0,92.0
4,2023-10-15,9793,832,140.0,94.0
5,2023-10-16,9500,788,129.0,61.0
6,2023-10-17,9088,780,127.0,44.0
7,2023-10-18,7664,652,94.0,62.0
8,2023-10-19,8434,697,120.0,77.0
9,2023-10-20,10496,860,153.0,98.0


In [79]:
# Split df_date into 4 weeks
bins = 4

for i in dfs:
    i['week_category'] = pd.cut(i['date'], bins=bins, labels=[1, 2, 3, 4])

display(df_control)

,date,pageviews,clicks,enrollments,payments,week_category
0,2023-10-11,7723,687,134.0,70.0,1
1,2023-10-12,9102,779,147.0,70.0,1
2,2023-10-13,10511,909,167.0,95.0,1
3,2023-10-14,9871,836,156.0,105.0,1
4,2023-10-15,10014,837,163.0,64.0,1
5,2023-10-16,9670,823,138.0,82.0,1
6,2023-10-17,9008,748,146.0,76.0,1
7,2023-10-18,7434,632,110.0,70.0,1
8,2023-10-19,8459,691,131.0,60.0,1
9,2023-10-20,10667,861,165.0,97.0,1


# Add metrics to A/B test. Cant just do views or clicks.

In [110]:
# Add CTRs
# Add Enrollment/Click
# Add Payment/Click
for df in dfs:
    df['ctr'] = df['clicks']/df['pageviews']
    df['enrollments_per_click'] = df['enrollments']/df['clicks']
    df['conversion_of_clicks'] = df['payments']/df['clicks']
    df['conversion_free_trial'] = df['payments']/df['enrollments']

In [112]:
df_control[:10]

,date,pageviews,clicks,enrollments,payments,week_category,ctr,enrollments_per_click,conversion_of_clicks,conversion_free_trial
0,2023-10-11,7723,687,134.0,70.0,1,0.088955,0.195051,0.101892,0.522388
1,2023-10-12,9102,779,147.0,70.0,1,0.085586,0.188703,0.089859,0.476190
2,2023-10-13,10511,909,167.0,95.0,1,0.086481,0.183718,0.104510,0.568862
3,2023-10-14,9871,836,156.0,105.0,1,0.084693,0.186603,0.125598,0.673077
4,2023-10-15,10014,837,163.0,64.0,1,0.083583,0.194743,0.076464,0.392638
5,2023-10-16,9670,823,138.0,82.0,1,0.085109,0.167679,0.099635,0.594203
6,2023-10-17,9008,748,146.0,76.0,1,0.083037,0.195187,0.101604,0.520548
7,2023-10-18,7434,632,110.0,70.0,1,0.085015,0.174051,0.110759,0.636364
8,2023-10-19,8459,691,131.0,60.0,1,0.081688,0.189580,0.086831,0.458015
9,2023-10-20,10667,861,165.0,97.0,1,0.080716,0.191638,0.112660,0.587879


In [113]:
# Add baseline as a column for each row.
for df in dfs:
    df['baseline_views'] = 40000
    df['baseline_clicks'] = 3200
    df['baseline_ctr'] = 0.08
    df['baseline_enrollments'] = 660
    df['baseline_enrollment_per_click'] = 0.20625
    df['baseline_conversion_free_trial'] = 0.53
    df['baseline_conversion_of_clicks'] = 0.0075

In [114]:
df_control[:5]

,date,pageviews,clicks,enrollments,payments,week_category,ctr,enrollments_per_click,conversion_of_clicks,conversion_free_trial,baseline_views,baseline_clicks,baseline_ctr,baseline_enrollments,baseline_enrollment_per_click,baseline_conversion_free_trial,baseline_conversion_of_clicks
0,2023-10-11,7723,687,134.0,70.0,1,0.088955,0.195051,0.101892,0.522388,40000,3200,0.08,660,0.20625,0.53,0.0075
1,2023-10-12,9102,779,147.0,70.0,1,0.085586,0.188703,0.089859,0.476190,40000,3200,0.08,660,0.20625,0.53,0.0075
2,2023-10-13,10511,909,167.0,95.0,1,0.086481,0.183718,0.104510,0.568862,40000,3200,0.08,660,0.20625,0.53,0.0075
3,2023-10-14,9871,836,156.0,105.0,1,0.084693,0.186603,0.125598,0.673077,40000,3200,0.08,660,0.20625,0.53,0.0075
4,2023-10-15,10014,837,163.0,64.0,1,0.083583,0.194743,0.076464,0.392638,40000,3200,0.08,660,0.20625,0.53,0.0075


In [118]:
# Do we have to divide the nominal baseline by 30 (monthly -> daily) or by 7 (weekly -> daily)?
# Both don't make sense vs. the 10k daily number above.
print(40000/30)
40000/7

1333.3333333333333


5714.285714285715

In [121]:
df_control.columns

Index(['date', 'pageviews', 'clicks', 'enrollments', 'payments',
       'week_category', 'ctr', 'enrollments_per_click', 'conversion_of_clicks',
       'conversion_free_trial', 'baseline_views', 'baseline_clicks',
       'baseline_ctr', 'baseline_enrollments', 'baseline_enrollment_per_click',
       'baseline_conversion_free_trial', 'baseline_conversion_of_clicks'],
      dtype='object')

In [129]:
cols_of_interest = ['week_category', 'ctr', 'enrollments_per_click', 'conversion_of_clicks',
       'conversion_free_trial']

In [130]:
# Aggregate by AVERAGES.
# Statistical tests are tests between 2 MEANS, not sums.
df_control_agg = df_control[cols_of_interest].groupby('week_category').mean()
df_experiment_agg = df_experiment[cols_of_interest].groupby('week_category').mean()
display(df_control_agg)
df_experiment_agg

C:\Users\kevin\AppData\Local\Temp\ipykernel_33072\1768384025.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_control_agg = df_control[cols_of_interest].groupby('week_category').mean()
C:\Users\kevin\AppData\Local\Temp\ipykernel_33072\1768384025.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_experiment_agg = df_experiment[cols_of_interest].groupby('week_category').mean()


,ctr,enrollments_per_click,conversion_of_clicks,conversion_free_trial
week_category,,,,
1,0.084486,0.186695,0.100981,0.543016
2,0.079264,0.244534,0.136878,0.559357
3,0.081550,0.250078,0.119621,0.487740
4,0.082955,NaN,NaN,NaN


,ctr,enrollments_per_click,conversion_of_clicks,conversion_free_trial
week_category,,,,
1,0.084497,0.162077,0.093242,0.575230
2,0.078743,0.219683,0.125179,0.573362
3,0.082158,0.248029,0.137136,0.567414
4,0.083108,NaN,NaN,NaN


In [135]:
# Nulls start in week 3.
df_experiment[df_experiment['week_category'] == 2]

,date,pageviews,clicks,enrollments,payments,week_category,ctr,enrollments_per_click,conversion_of_clicks,conversion_free_trial,baseline_views,baseline_clicks,baseline_ctr,baseline_enrollments,baseline_enrollment_per_click,baseline_conversion_free_trial,baseline_conversion_of_clicks
10,2023-10-21,10551,864,143.0,71.0,2,0.081888,0.165509,0.082176,0.496503,40000,3200,0.08,660,0.20625,0.53,0.0075
11,2023-10-22,9737,801,128.0,70.0,2,0.082264,0.159800,0.087391,0.546875,40000,3200,0.08,660,0.20625,0.53,0.0075
12,2023-10-23,8176,642,122.0,68.0,2,0.078523,0.190031,0.105919,0.557377,40000,3200,0.08,660,0.20625,0.53,0.0075
13,2023-10-24,9402,697,194.0,94.0,2,0.074133,0.278336,0.134864,0.484536,40000,3200,0.08,660,0.20625,0.53,0.0075
14,2023-10-25,8669,669,127.0,81.0,2,0.077172,0.189836,0.121076,0.637795,40000,3200,0.08,660,0.20625,0.53,0.0075
15,2023-10-26,8881,693,153.0,101.0,2,0.078032,0.220779,0.145743,0.660131,40000,3200,0.08,660,0.20625,0.53,0.0075
16,2023-10-27,9655,771,213.0,119.0,2,0.079855,0.276265,0.154345,0.558685,40000,3200,0.08,660,0.20625,0.53,0.0075
17,2023-10-28,9396,736,162.0,120.0,2,0.078331,0.220109,0.163043,0.740741,40000,3200,0.08,660,0.20625,0.53,0.0075
18,2023-10-29,9262,727,201.0,96.0,2,0.078493,0.276479,0.132050,0.477612,40000,3200,0.08,660,0.20625,0.53,0.0075


In [133]:
# See if the means are significant by week.
# This is not conversion (0, 1) so we use a t/z test instead of a chi squared test.
# Do a T Test.
# Split it up by week.

# Remove 'week_category'
cols_of_interest = ['ctr', 'enrollments_per_click', 'conversion_of_clicks',
       'conversion_free_trial']

for week in range(1, 5):
    print("Week " + str(week))
    for i in cols_of_interest:
        t_statistic, p_value = stats.ttest_ind(df_control[df_control['week_category'] <= week][i], 
                                               df_experiment[df_experiment['week_category'] <= week][i])
        print(i)
        print("T-statistic:", t_statistic)
        print("P-value:", p_value)


Week 1
ctr
T-statistic: -0.011479095526949172
P-value: 0.9909674809214706
enrollments_per_click
T-statistic: 5.503216820571179
P-value: 3.166186538237567e-05
conversion_of_clicks
T-statistic: 0.861337426799341
P-value: 0.400381199449678
conversion_free_trial
T-statistic: -0.5878073809385489
P-value: 0.5639640281741107
Week 2
ctr
T-statistic: 0.19673443816473993
P-value: 0.8451418341354349
enrollments_per_click
T-statistic: 1.7271150071580434
P-value: 0.09271934639872713
conversion_of_clicks
T-statistic: 0.9618030643101384
P-value: 0.3425646607427959
conversion_free_trial
T-statistic: -0.6879569516949988
P-value: 0.49588905732416344
Week 3
ctr
T-statistic: -0.03595561740325199
P-value: 0.9714503358303426
enrollments_per_click
T-statistic: nan
P-value: nan
conversion_of_clicks
T-statistic: nan
P-value: nan
conversion_free_trial
T-statistic: nan
P-value: nan
Week 4
ctr
T-statistic: -0.08302608446561602
P-value: 0.9340611962072063
enrollments_per_click
T-statistic: nan
P-value: nan
convers

**Inconclusive**

1. We have 2.5 weeks of data. Out of 4.
2.  The p value is pretty high for all 4 rates we're measuring here. The means of the ctr and conversion rates are really not that different from one another. At the 10% sig level, we could say that enrollments per click is stat sig. But it is not really that convincing since we use 5% most of the time as the baseline. It is also stat sig in the wrong direction, but that is what we are looking for. Less free trials signups from students that can't commit the time. You might think that since we have more committed students taking the free trial, it would result in a higher conversion rate to paid users. But that did not seem to happen either.
